## Imports

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import json
import re
from transformers import AutoTokenizer
pd.set_option('display.max_colwidth', None)

# Functions

In [ ]:
def read_file_txt(file_path)->str:

  with open(file_path, "r", encoding="utf-8") as file:
    content = file.read()

  return content

def count_ids(content: str) -> None:
    """
    Conta IDs com sufixos específicos (ins, inp, out) no conteúdo fornecido.

    Args:
        content (str): O texto que contém os IDs a serem contados.

    Prints:
        Contagem de IDs por sufixo e o total de IDs.
    """
    suffixes = ["ins", "inp", "out"]
    counts = {}

    for suffix in suffixes:
        pattern = fr'[\'"]ID[\'"]:\s*[\'"]\d+{suffix}[\'"]' #     pattern = fr'"ID":\s*"\d+{suffix}"' pega menos id
        matches = re.findall(pattern, content)
        counts[suffix] = len(matches)
        print(f"Total de IDs com sufixo '{suffix}': {counts[suffix]}")

    total_ids = sum(counts.values())
    print("Total ids:", total_ids)

def extrair_dados_multiplos(texto):
    """
    Extrai dados estruturados de um texto baseado em um padrão de expressão regular,
    considerando aspas simples ou duplas nos campos.
    """
    # Padrão ajustado para aceitar aspas simples ou duplas
    padrao = re.compile(
        r"[\'\"]ID[\'\"]:\s*[\'\"]([^\'\"]+)[\'\"]\s*,\s*[\'\"]ORIGINAL[\'\"]:\s*[\'\"](.*?)(?<!\\)[\'\"]\s*,\s*[\'\"]FIGURATIVO[\'\"]:\s*(null|[\'\"](.*?)(?<!\\)[\'\"])\s*,\s*[\'\"]REESCRITO[\'\"]:\s*(null|[\'\"](.*?)(?<!\\)[\'\"])\s*,\s*[\'\"]RESULTADO[\'\"]:\s*[\'\"](.*?)(?<!\\)[\'\"]",
        re.DOTALL
    )

    matches = padrao.findall(texto)
    resultados = []

    for match in matches:
        id_val = match[0]
        original_val = match[1]
        figurativo_val = match[3] if match[2] != 'null' else None
        reescrito_val = match[5] if match[4] != 'null' else None
        resultado_val = match[6]

        resultados.append((id_val, original_val, figurativo_val, reescrito_val, resultado_val))

    return resultados

def processar_content_para_dataframe(content):

    ids, originais, figurativos, reescritos, resultados = [], [], [], [], []

    blocos_dados = content.split("json\n[\n")
    blocos_dados = [bloco.replace('\u200b', '').strip() for bloco in blocos_dados]

    for bloco in blocos_dados:
        conjuntos = extrair_dados_multiplos(bloco)
        for id_val, original_val, figurativo_val, reescrito_val, resultado_val in conjuntos:
            if id_val and original_val and resultado_val:
                ids.append(id_val)
                originais.append(original_val)
                figurativos.append(figurativo_val)
                reescritos.append(reescrito_val)
                resultados.append(resultado_val)

    df_result = pd.DataFrame({
        'ID': ids,
        'ORIGINAL': originais,
        'FIGURATIVO': figurativos,
        'REESCRITO': reescritos,
        'RESULTADO': resultados
    })

    print(f"DataFrame - shape: {df_result.shape}, IDs únicos: {df_result.ID.nunique()}")

    return df_result

def drop_duplicates_by_count_col(df, col):

  df['COUNT_'+col]= df[col].apply(lambda x: None if x==None else len(x))
  df = df.sort_values(by='COUNT_'+col, ascending=False)
  df = df.drop_duplicates(subset='ID', keep='first')
  df = df.sort_values(by='ID', ascending=True)

  print(f"After drop\nDataFrame - shape: {df.shape}, IDs únicos: {df.ID.nunique()}")

  return df

def process_dataframe_to_base(df, ins_column, inp_column, out_column):
    """
    Processa um DataFrame para criar um novo DataFrame base com as colunas especificadas.

    Parâmetros:
        df (pd.DataFrame): DataFrame original.
        ins_column (str): Nome da coluna para 'INSTRUCTION'.
        inp_column (str): Nome da coluna para 'INPUT'.
        out_column (str): Nome da coluna para 'OUTPUT'.

    Retorna:
        pd.DataFrame: DataFrame base com colunas 'ID', 'INSTANCE', e 'CLASS_TYPE'.
    """
    df_base = df.copy()

    # Criando o DataFrame ins
    ins = df_base[['ID', ins_column]].copy()
    ins['CLASS_TYPE'] = 'ins'
    ins['ID'] = ins['ID'].astype(str) + ins['CLASS_TYPE']
    ins = ins.rename(columns={ins_column: 'INSTANCE'})

    # Criando o DataFrame inp
    inp = df_base[['ID', inp_column]].copy()
    inp['CLASS_TYPE'] = 'inp'
    inp['ID'] = inp['ID'].astype(str) + inp['CLASS_TYPE']
    inp = inp.rename(columns={inp_column: 'INSTANCE'})

    # Criando o DataFrame out
    out = df_base[['ID', out_column]].copy()
    out['CLASS_TYPE'] = 'out'
    out['ID'] = out['ID'].astype(str) + out['CLASS_TYPE']
    out = out.rename(columns={out_column: 'INSTANCE'})

    # Concatenando os DataFrames
    df_base = pd.concat([ins, inp, out], axis=0)[['ID', 'INSTANCE', 'CLASS_TYPE']]

    return df_base

# Base to figurative

São os arquivos que vão para o codigo .py e consequentemente para o gemini.

['ID', 'INSTANCE', 'CLASS_TYPE', 'TOKENS_EST_INSTANCE', 'TOKENS_EST_ID']

In [ ]:
df_base = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/DATASETS/GEMINI/INPUT/alpaca_to_identify_figurative.csv")
df_base.columns = df_base.columns.str.upper()
df_base['ID'] = df_base['ID'].str.replace("_", "")
df_base.head(6)

,ID,INSTANCE,CLASS_TYPE,TOKENS_EST_INSTANCE,TOKENS_EST_ID
0,0ins,Dê três dicas para se manter saudável.,ins,16,3
1,2ins,Descreva a estrutura de um átomo.,ins,15,3
2,3ins,Como podemos reduzir a poluição do ar?,ins,16,3
3,4ins,Descreva uma ocasião em que você teve que tomar uma decisão difícil.,ins,28,3
4,5ins,Identifique o estranho.,ins,8,3
5,6ins,Explique por que a seguinte fração é equivalente a 1/4,ins,19,3


# Load Results

In [ ]:
base_path = "/content/drive/MyDrive/Colab Notebooks/DATASETS/FIGURATIVE/"

print("Single---")
content_single = read_file_txt(base_path+"single/single_combined.txt")
count_ids(content_single)
df_single = processar_content_para_dataframe(content_single)
df_single = drop_duplicates_by_count_col(df_single, 'RESULTADO')


print("\nBucket---")
content_bucket = read_file_txt(base_path+"bucket/bucket_combined.txt")
count_ids(content_bucket)
df_bucket = processar_content_para_dataframe(content_bucket)
df_bucket = drop_duplicates_by_count_col(df_bucket, 'RESULTADO')

Single---
Total de IDs com sufixo 'ins': 1690
Total de IDs com sufixo 'inp': 794
Total de IDs com sufixo 'out': 52470
Total ids: 54954
DataFrame - shape: (54824, 5), IDs únicos: 27356
After drop
DataFrame - shape: (27356, 6), IDs únicos: 27356

Bucket---
Total de IDs com sufixo 'ins': 250223
Total de IDs com sufixo 'inp': 86589
Total de IDs com sufixo 'out': 216442
Total ids: 553254
DataFrame - shape: (548570, 5), IDs únicos: 119124
After drop
DataFrame - shape: (119124, 6), IDs únicos: 119124


In [ ]:
df_figurative = pd.concat(axis= 0, objs=[df_single, df_bucket])
df_figurative = drop_duplicates_by_count_col(df_figurative, 'RESULTADO')
print(f"Joined single and bucket: {df_figurative.shape}, ids únicos {df_figurative.ID.nunique()}")
df_figurative.head(2)

After drop
DataFrame - shape: (119124, 6), IDs únicos: 119124
Joined single and bucket: (119124, 6), ids únicos 119124


,ID,ORIGINAL,FIGURATIVO,REESCRITO,RESULTADO,COUNT_RESULTADO
355308,0ins,Dê três dicas para se manter saudável.,None,None,Dê três dicas para se manter saudável.,38
295744,0out,1. Coma uma dieta balanceada e certifique-se de incluir muitas frutas e vegetais. \n2. Exercite-se regularmente para manter seu corpo ativo e forte. \n3. Durma o suficiente e mantenha um horário de sono consistente.,None,None,1. Coma uma dieta balanceada e certifique-se de incluir muitas frutas e vegetais. \n2. Exercite-se regularmente para manter seu corpo ativo e forte. \n3. Durma o suficiente e mantenha um horário de sono consistente.,215


In [ ]:
df_figurative.query("ID == '6ins'")

,ID,ORIGINAL,FIGURATIVO,REESCRITO,RESULTADO,COUNT_RESULTADO
437749,6ins,Explique por que a seguinte fração é equivalente a 1/4,None,None,Explique por que a seguinte fração é equivalente a 1/4,54


In [ ]:
df_figurative = df_figurative[['ID', 'RESULTADO', 'COUNT_RESULTADO']]
df_figurative.head()

,ID,RESULTADO,COUNT_RESULTADO
355308,0ins,Dê três dicas para se manter saudável.,38
295744,0out,1. Coma uma dieta balanceada e certifique-se de incluir muitas frutas e vegetais. \n2. Exercite-se regularmente para manter seu corpo ativo e forte. \n3. Durma o suficiente e mantenha um horário de sono consistente.,215
243564,10000ins,Cite cinco países do Mediterrâneo.,34
186455,10000out,"Grécia, Itália, Espanha, Turquia, França",40
51199,10001inp,Qual dos seguintes alimentos contém mais açúcar?\nA. Maçã\nB. Laranja\nC. Chocolate,83


In [ ]:
df_figurative.query('ID == "51834out"')

,ID,RESULTADO,COUNT_RESULTADO


# Selecting not figuratived :

Erros que não fizeram o processo de identificação dos figurativos

In [ ]:
not_figuratived = df_base[~df_base['ID'].isin(df_figurative['ID'].unique())]
not_figuratived

,ID,INSTANCE,CLASS_TYPE,TOKENS_EST_INSTANCE,TOKENS_EST_ID
2372,2374ins,Descreva o que é um buraco negro,ins,10,4
9375,9378ins,O que é um evento Cisne Negro?,ins,11,4
10156,10160ins,Crie uma analogia para ilustrar o conceito de buraco negro.,ins,18,4
16879,16885ins,"Crie uma história de terror de três frases que incorpore ""corvos negros"" e ""medos supersticiosos"".",ins,35,4
17758,17764ins,"Como você descreveria o conceito de ""buraco negro""?",ins,17,4
...,...,...,...,...,...
116127,49559out,"Um buraco negro é uma região no espaço onde a gravidade é tão intensa que nada, nem mesmo a luz, pode escapar de sua atração. Eles são formados quando uma grande estrela morre e colapsa sobre si mesma, formando um objeto incrivelmente denso, muito menor do que seu tamanho original.",out,104,4
116140,49573out,"""A solidão expressa a dor de estar sozinho e a solitude expressa a glória de estar sozinho."" ― Paul Tillich",out,35,4
116213,49652out,1. Leão\n2. Elefante africano\n3. Girafa\n4. Cão selvagem africano\n5. Rinoceronte negro,out,38,4
117066,50564out,1. O Silêncio dos Inocentes \n2. Ilha do Medo \n3. Psicose \n4. Cisne Negro \n5. Amnésia,out,43,4


# Formating to GLOSA translate

In [ ]:
def calculate_tokens(df, columns, tokenizer_model="gpt2"):
    """
    Calcula o número de tokens para colunas especificadas em um DataFrame usando um tokenizer.

    Args:
        df (pd.DataFrame): DataFrame de entrada contendo os dados de texto.
        columns (list): Lista de colunas do DataFrame para calcular os tokens.
        tokenizer_model (str): Nome do modelo para carregar o tokenizer. Padrão é "gpt2".

    Returns:
        pd.DataFrame: DataFrame atualizado com colunas contendo o número de tokens.
        pd.DataFrame: DataFrame contendo registros com erros.
    """
    # Carregar o tokenizer
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_model)

    # Inicializar lista para armazenar erros
    error_records = []

    # Função auxiliar para calcular tokens
    def count_tokens(text, index):
        if isinstance(text, str):  # Verifica se o texto é uma string
            return len(tokenizer.encode(text, truncation=True))
        else:
            error_records.append({'index': index, 'value': text})
            return 0

    # Criar novas colunas para o número de tokens
    for column in columns:
        token_column_name = f"TOKENS_EST_{column.upper()}"
        df[token_column_name] = df.apply(lambda row: count_tokens(row[column], row.name), axis=1)

    # Criar DataFrame de erros
    error_df = pd.DataFrame(error_records)

    # Exibir mensagem sobre erros
    if error_df.empty:
        print("Nenhum erro encontrado.")
    else:
        print("Erros encontrados:")
        print(error_df)

    return df, error_df

columns_to_tokenize = ['ID', 'INSTANCE']
to_gloss, error_df = calculate_tokens(not_figuratived, columns_to_tokenize)
to_gloss.head()
# salvar em um dataframe depois

Erros encontrados:
   index  value
0  65719    NaN


<ipython-input-124-e551c5cce845>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[token_column_name] = df.apply(lambda row: count_tokens(row[column], row.name), axis=1)


,ID,INSTANCE,CLASS_TYPE,TOKENS_EST_INSTANCE,TOKENS_EST_ID
2372,2374ins,Descreva o que é um buraco negro,ins,10,3
9375,9378ins,O que é um evento Cisne Negro?,ins,11,3
10156,10160ins,Crie uma analogia para ilustrar o conceito de buraco negro.,ins,18,3
16879,16885ins,"Crie uma história de terror de três frases que incorpore ""corvos negros"" e ""medos supersticiosos"".",ins,35,3
17758,17764ins,"Como você descreveria o conceito de ""buraco negro""?",ins,17,3


# Not figuratived by translated originals

In [ ]:
df_translated = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/DATASETS/alpaca_translated.csv")
df_translated.columns = df_translated.columns.str.upper()
df_translated = df_translated.rename(columns={'ORIGINAL_INDEX':'ID', 'INSTRUCAO':'INSTRUCTION', 'ENTRADA':'INPUT', 'SAIDA':'OUTPUT'})
df_translated = df_translated[['ID','INSTRUCTION', 'INPUT', 'OUTPUT']]
df_translated.shape

(52002, 4)

In [ ]:
def process_dataframe_to_base(df, ins_column, inp_column, out_column):
    """
    Processa um DataFrame para criar um novo DataFrame base com as colunas especificadas.

    Parâmetros:
        df (pd.DataFrame): DataFrame original.
        ins_column (str): Nome da coluna para 'INSTRUCTION'.
        inp_column (str): Nome da coluna para 'INPUT'.
        out_column (str): Nome da coluna para 'OUTPUT'.

    Retorna:
        pd.DataFrame: DataFrame base com colunas 'ID', 'INSTANCE', e 'CLASS_TYPE'.
    """
    df_base = df.copy()

    # Criando o DataFrame ins
    ins = df_base[['ID', ins_column]].copy()
    ins['CLASS_TYPE'] = 'ins'
    ins['ID'] = ins['ID'].astype(str) + ins['CLASS_TYPE']
    ins = ins.rename(columns={ins_column: 'INSTANCE'})

    # Criando o DataFrame inp
    inp = df_base[['ID', inp_column]].copy()
    inp['CLASS_TYPE'] = 'inp'
    inp['ID'] = inp['ID'].astype(str) + inp['CLASS_TYPE']
    inp = inp.rename(columns={inp_column: 'INSTANCE'})

    # Criando o DataFrame out
    out = df_base[['ID', out_column]].copy()
    out['CLASS_TYPE'] = 'out'
    out['ID'] = out['ID'].astype(str) + out['CLASS_TYPE']
    out = out.rename(columns={out_column: 'INSTANCE'})

    # Concatenando os DataFrames
    df_base = pd.concat([ins, inp, out], axis=0)[['ID', 'INSTANCE', 'CLASS_TYPE']]

    return df_base

df_translated = process_dataframe_to_base(df_translated, 'INSTRUCTION', 'INPUT', 'OUTPUT')
df_translated.head()

,ID,INSTANCE,CLASS_TYPE
0,0ins,Dê três dicas para se manter saudável.,ins
1,1ins,Quais são as três cores primárias?,ins
2,2ins,Descreva a estrutura de um átomo.,ins
3,3ins,Como podemos reduzir a poluição do ar?,ins
4,4ins,Descreva um momento em que você teve que tomar uma decisão difícil.,ins


In [ ]:
df_erros = df_translated[~df_translated['ID'].isin(df_base['ID'].unique())]
df_erros.shape

(37588, 3)

In [ ]:
df_erros.head()

,ID,INSTANCE,CLASS_TYPE
1,1ins,Quais são as três cores primárias?,ins
11,11ins,Qual é a capital da França?,ins
8441,8441ins,Quais são os benefícios de aprender uma língua estrangeira?,ins
9748,9748ins,Compare esses dois itens.,ins
12040,12040ins,Qual é a cor do céu?,ins


## Functions Specials Instances

In [ ]:
import pandas as pd
import re

# Funções para detectar categorias
def code_detector(textoVerificar):
    """
    Verifica se o texto contém padrões comuns de código.
    """
    padrao_codigo = r"""(
        \bdef\b|
        \bclass\b|
        \belse\b|
        \breturn\b|
        \bprint\b|
        \bimport\b|
        \blambda\b|
        \belif\b|
        \basync\b|
        \bfunction\b|
        \bconst\b|
        \bvar\b|
        \bstatic\b|
        \bvoid\b|
        \bloop\b|
        \benum\b|
        \bimpl\b|
        \bcout|
        \bcin|
        \bnamespace\b|
        std|
        printf|
        System\.Out\.Print|
        System\.Out\.Println|
        \++|
        \--|
        \+=|
        \-=|
        \*=|
        \/=|
        \%=|
        \==|
        \===|
        \!=|
        \!==|
        \>=|
        \<=|
        \&&|
        \||
    )"""
    return bool(re.search(padrao_codigo, str(textoVerificar)))

# Outras funções de categoria
# Certifique-se de que estas funções já estão definidas:

def contains_image_or_link(text: str) -> bool:
    image_link_pattern = r"\b\w+\.(png|jpg|jpeg|com)\b"
    return bool(re.search(image_link_pattern, text, re.IGNORECASE))

def is_all_numeric(text: str) -> bool:
    text = text.strip()
    if text.startswith('[') and text.endswith(']'):
        try:
            eval_list = eval(text)
            if isinstance(eval_list, list):
                for item in eval_list:
                    if not is_all_numeric(str(item)):
                        return False
                return True
        except:
            return False
    if re.search(r'[a-zA-Z]{2,}', text):
        return False
    math_variable_pattern = r'^[\d\s\+\-\*/\(\)\.,=a-zA-Z]*$'
    return bool(re.match(math_variable_pattern, text))

def contains_only_specials_or_emojis(text: str) -> bool:
    return bool(re.match(r'^[^\w\s]+$', text))

def contains_image_or_blob(text: str) -> bool:
    return any(keyword in str(text) for keyword in ['Blob', 'Image', 'IPython.display.Image'])

# Função principal
def check_instance_category(text: str) -> bool:

    text = str(text)
    if code_detector(text):
        return True
    if contains_image_or_link(text):
        return True
    if is_all_numeric(text):
        return True
    if contains_only_specials_or_emojis(text):
        return True
    if contains_image_or_blob(text):
        return True
    return False

# Aplicando ao DataFrame
df_erros['INSTANCE'] = df_erros['INSTANCE'].astype(str)
df_erros['SPECIAL'] = df_erros['INSTANCE'].apply(check_instance_category)

<ipython-input-137-e04f048a8b87>:99: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_erros['INSTANCE'] = df_erros['INSTANCE'].astype(str)
<ipython-input-137-e04f048a8b87>:100: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_erros['SPECIAL'] = df_erros['INSTANCE'].apply(check_instance_category)


In [ ]:
df_erros.head()

,ID,INSTANCE,CLASS_TYPE,SPECIAL,CATEGORY
1,1ins,Quais são as três cores primárias?,ins,False,[code]
11,11ins,Qual é a capital da França?,ins,False,[code]
8441,8441ins,Quais são os benefícios de aprender uma língua estrangeira?,ins,False,[code]
9748,9748ins,Compare esses dois itens.,ins,False,[code]
12040,12040ins,Qual é a cor do céu?,ins,False,[code]


In [ ]:
def detect_categories(text):
    """
    Detecta as categorias que se aplicam ao texto.
    Retorna uma lista com as categorias detectadas.
    """
    text = str(text)  # Garantir que o texto seja uma string
    categories = []

    if code_detector(text):
        categories.append('code')
    if contains_image_or_link(text):
        categories.append('image_or_link')
    if is_all_numeric(text):
        categories.append('all_numeric')
    if contains_only_specials_or_emojis(text):
        categories.append('special_or_emoji')
    if contains_image_or_blob(text):
        categories.append('blob_or_image')

    return categories

# Aplicar a detecção ao DataFrame
df_erros['INSTANCE'] = df_erros['INSTANCE'].astype(str)  # Garantir que seja string
df_erros['CATEGORY'] = df_erros['INSTANCE'].apply(detect_categories)
df_erros['SPECIAL'] = df_erros['CATEGORY'].apply(lambda x: len(x) > 0)  # True se houver categorias detectadas


<ipython-input-132-95ff78f26cb7>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_erros['INSTANCE'] = df_erros['INSTANCE'].astype(str)  # Garantir que seja string
<ipython-input-132-95ff78f26cb7>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_erros['CATEGORY'] = df_erros['INSTANCE'].apply(detect_categories)
<ipython-input-132-95ff78f26cb7>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the

In [ ]:
df_erros.head()

,ID,INSTANCE,CLASS_TYPE,SPECIAL,CATEGORY
1,1ins,Quais são as três cores primárias?,ins,True,[code]
11,11ins,Qual é a capital da França?,ins,True,[code]
8441,8441ins,Quais são os benefícios de aprender uma língua estrangeira?,ins,True,[code]
9748,9748ins,Compare esses dois itens.,ins,True,[code]
12040,12040ins,Qual é a cor do céu?,ins,True,[code]


In [ ]:
df_erros['CATEGORY'].value_counts()

,count
CATEGORY,
[code],35568
"[code, all_numeric]",1973
"[code, image_or_link]",42
"[code, blob_or_image]",4
"[code, image_or_link, blob_or_image]",1


In [ ]:
df_erros['SPECIAL'] = df_erros['INSTANCE'].apply(check_instance_category)
df_erros.head()

TypeError: 'bool' object is not subscriptable

# Formating in original chat

In [ ]:
def format_result_figurative(df):
    """
    Formata o DataFrame fornecido para ajustar os campos relacionados a resultados figurativos.

    Args:
        df (pd.DataFrame): DataFrame de entrada contendo a coluna 'INSTANCE'.

    Returns:
        pd.DataFrame: DataFrame formatado com colunas específicas e valores ajustados.
    """
    print(df.shape, df['ID'].nunique())

    # Adicionando colunas adicionais
    df['ORIGINAL'] = None
    df['FIGURATIVO'] = None
    df['REESCRITO'] = None

    # Calculando o número de itens em 'INSTANCE' e renomeando a coluna
    df['COUNT_RESULTADO'] = df['INSTANCE'].apply(
        lambda x: None if x is None or not isinstance(x, (list, str)) else len(x)
    )
    df = df.rename(columns={'INSTANCE': 'RESULTADO'})

    # Selecionando colunas de interesse
    df = df[['ID', 'ORIGINAL', 'FIGURATIVO', 'REESCRITO', 'RESULTADO', 'COUNT_RESULTADO']]

    print(df.shape, df['ID'].nunique())

    return df

df_not_figuratived = format_result_figurative(df_not_figuratived)
df_not_figuratived.head()

(7386, 3) 7386
(7386, 6) 7386


,ID,ORIGINAL,FIGURATIVO,REESCRITO,RESULTADO,COUNT_RESULTADO
0,6ins,None,None,None,NaN,NaN
1,17ins,None,None,None,NaN,NaN
2,37ins,None,None,None,NaN,NaN
3,41ins,None,None,None,NaN,NaN
4,53ins,None,None,None,NaN,NaN


In [ ]:
df_not_figuratived.query("ID == '6ins'")

,ID,ORIGINAL,FIGURATIVO,REESCRITO,RESULTADO,COUNT_RESULTADO
0,6ins,None,None,None,NaN,NaN


In [ ]:
df_figurative[df_figurative['ID'].isin(df_not_figuratived['ID'])]

,ID,ORIGINAL,FIGURATIVO,REESCRITO,RESULTADO,COUNT_RESULTADO


In [ ]:
#Join com o Figurative
df_figurative_complete = pd.concat([df_figurative, df_not_figuratived], axis=0)

df_figurative_complete = df_figurative_complete.dropna(subset=['ORIGINAL', 'FIGURATIVO', 'REESCRITO', 'RESULTADO', 'COUNT_RESULTADO'], how='all')

print(df_figurative_complete.shape, df_figurative_complete.ID.nunique())

(122150, 6) 122150


In [ ]:
df_base.shape

(118418, 5)

In [ ]:
(52002 * 3) == 118418

False

In [ ]:
print(f"Base-\n shape: {df_base.shape}\n ids únicos: {df_base.ID.nunique()}")

Base-
 shape: (118418, 5)
 ids únicos: 118418


In [ ]:
ids_uniques = df_base.ID.nunique()
ids_uniques

118418

# Join figurative responses


Juntar todos os arquivos que foram passados para o gemini

In [ ]:
base_path = "/content/drive/MyDrive/Colab Notebooks/DATASETS/GEMINI/OUTPUT/FIGURATIVE/"

input_paths = [
    base_path+"figurative1_response.txt",
    base_path+"figurative2_response.txt",
    base_path+"figurative3_response.txt",
    base_path+"figurative4_response.txt",
    base_path+"figurative5_response.txt",
    base_path+"figurative6_response.txt",
    base_path+"figurative7_response.txt",
    base_path+"figurative_erros_response.txt"
]

output_path = base_path+"figurative_response_joined.txt"

line_counts = []

with open(output_path, 'w', encoding='utf-8') as output_file:
    for file_path in input_paths:
        with open(file_path, 'r', encoding='utf-8') as input_file:
            lines = input_file.readlines()
            line_counts.append(len(lines))
            output_file.write("".join(lines) + "\n")


with open(output_path, 'r', encoding='utf-8') as output_file:
    count_merged = sum(1 for _ in output_file)

for i, count in enumerate(line_counts, 1):
    print(f"Number of lines in file {i}: {count}")
print(f"Number of lines in the merged file: {count_merged}")

Number of lines in file 1: 586669
Number of lines in file 2: 182168
Number of lines in file 3: 45104
Number of lines in file 4: 3094
Number of lines in file 5: 12401
Number of lines in file 6: 21756
Number of lines in file 7: 22
Number of lines in file 8: 212
Number of lines in the merged file: 851434


In [ ]:
'''Number of lines in file 1: 586669
Number of lines in file 2: 182168
Number of lines in file 3: 45104
Number of lines in file 4: 3094
Number of lines in file 5: 12401
Number of lines in file 6: 21756
Number of lines in file 7: 22
Number of lines in the merged file: 851221'''

'Number of lines in file 1: 586669\nNumber of lines in file 2: 182168\nNumber of lines in file 3: 45104\nNumber of lines in file 4: 3094\nNumber of lines in file 5: 12401\nNumber of lines in file 6: 21756\nNumber of lines in file 7: 22\nNumber of lines in the merged file: 851221'

## Leitura dos arquivos completos após join

In [ ]:
with open(output_path, 'r', encoding='utf-8') as file:
    content = file.read()

suffixes = ["ins", "inp", "out"]

counts = {}

for suffix in suffixes:
    pattern = fr'"ID":\s*"\d+{suffix}"'
    matches = re.findall(pattern, content)
    counts[suffix] = len(matches)
    print(f"Total de IDs com sufixo '{suffix}': {counts[suffix]}")

Total de IDs com sufixo 'ins': 52390
Total de IDs com sufixo 'inp': 17943
Total de IDs com sufixo 'out': 49008


In [ ]:
ids_uniques - (52390+17938+48994)# tem duplicados

-904

# Prepare csv to GLOSA format

Dataframe-
 shape: (118321, 5)
 ids únicos: 118321


,ID,ORIGINAL,FIGURATIVO,REESCRITO,RESULTADO
0,0ins,Dê três dicas para se manter saudável.,None,None,Dê três dicas para se manter saudável.
1,2ins,Descreva a estrutura de um átomo.,None,None,Descreva a estrutura de um átomo.
2,3ins,Como podemos reduzir a poluição do ar?,None,None,Como podemos reduzir a poluição do ar?
3,4ins,Descreva uma ocasião em que você teve que tomar uma decisão difícil.,None,None,Descreva uma ocasião em que você teve que tomar uma decisão difícil.
4,5ins,Identifique o estranho.,None,None,Identifique o estranho.


In [ ]:
df_result.query("ID == '12343ins'")

,ID,ORIGINAL,FIGURATIVO,REESCRITO,RESULTADO
10803,12343ins,Crie um novo sistema de gerenciamento de tempo para profissionais que trabalham.,None,None,Crie um novo sistema de gerenciamento de tempo para profissionais que trabalham.


# Prepare to GLOSS: tokens estimate

['ID', 'PT', 'TOKENS_ESTIMATE']

In [ ]:
df_to_glosa = df_result[['ID','RESULTADO']].rename( columns={'RESULTADO':'PT'})
print(f"Df glosa- \n shape: {df_to_glosa.shape}\n ids únicos: {df_to_glosa.ID.nunique()}")
df_to_glosa.head()

Df glosa- 
 shape: (118321, 2)
 ids únicos: 118321


,ID,PT
0,0ins,Dê três dicas para se manter saudável.
1,2ins,Descreva a estrutura de um átomo.
2,3ins,Como podemos reduzir a poluição do ar?
3,4ins,Descreva uma ocasião em que você teve que tomar uma decisão difícil.
4,5ins,Identifique o estranho.


In [ ]:
from transformers import AutoTokenizer
import pandas as pd

# Carregar o tokenizer do modelo GPT-2
tokenizer = AutoTokenizer.from_pretrained("gpt2")

# Inicializar uma lista para armazenar erros
error_records = []

# Função para calcular o número de tokens
def count_tokens(text, index):
    if isinstance(text, str):  # Verifica se o texto é uma string
        return len(tokenizer.encode(text, truncation=True))
    else:
        error_records.append({'index': index, 'PT': text})
        return 0

df_to_glosa['TOKENS_EST_INSTANCE'] = df_to_glosa.apply(lambda row: count_tokens(row['PT'], row.name), axis=1)

df_to_glosa['TOKENS_EST_ID'] = df_to_glosa.apply(lambda row: count_tokens(row['ID'], row.name), axis=1)

if len(error_records) == 0:
    print("Nenhum erro encontrado.")
else:
    print("Erros encontrados:")
    error_df = pd.DataFrame(error_records)
    print(error_df)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Nenhum erro encontrado.


In [ ]:
print(f"Df glosa- \n shape: {df_to_glosa.shape}\n ids únicos: {df_to_glosa.ID.nunique()}")
df_to_glosa.head()

Df glosa- 
 shape: (118321, 4)
 ids únicos: 118321


,ID,PT,TOKENS_EST_INSTANCE,TOKENS_EST_ID
0,0ins,Dê três dicas para se manter saudável.,16,2
1,2ins,Descreva a estrutura de um átomo.,15,2
2,3ins,Como podemos reduzir a poluição do ar?,16,2
3,4ins,Descreva uma ocasião em que você teve que tomar uma decisão difícil.,28,2
4,5ins,Identifique o estranho.,8,2


In [ ]:
df_to_glosa.to_csv('/content/drive/MyDrive/Colab Notebooks/DATASETS/GEMINI/INPUT/formated_glosa_to_input.csv', index=False)

# Filtering *errors* in base data

In [ ]:
# Verifica se todos os que estão em glosa realmente estão no base
df_base[df_base['ID'].isin(df_to_glosa.ID.tolist())].shape[0] == df_to_glosa.ID.nunique()

True

In [ ]:
# Filtra os que estão no base e não foram para o figurative, ou seja, eles não foram para a glosa
df_base[~df_base['ID'].isin(df_to_glosa.ID.tolist())].shape[0]

97

In [ ]:
108-97

11

In [ ]:
df_not_figuratived = df_base[~df_base['ID'].isin(df_to_glosa.ID.tolist())]
print(f"Erros em figurative-\n shape: df_not_figuratived: {df_not_figuratived.shape}\n ids únicos: {df_not_figuratived.ID.nunique()}")
df_not_figuratived.head()

Erros em figurative-
 shape: df_not_figuratived: (97, 5)
 ids únicos: 97


,ID,INSTANCE,CLASS_TYPE,TOKENS_EST_INSTANCE,TOKENS_EST_ID
2372,2374ins,Descreva o que é um buraco negro,ins,10,4
9375,9378ins,O que é um evento Cisne Negro?,ins,11,4
10156,10160ins,Crie uma analogia para ilustrar o conceito de buraco negro.,ins,18,4
16879,16885ins,"Crie uma história de terror de três frases que incorpore ""corvos negros"" e ""medos supersticiosos"".",ins,35,4
17758,17764ins,"Como você descreveria o conceito de ""buraco negro""?",ins,17,4


In [ ]:
#df_not_figuratived.to_csv("/content/drive/MyDrive/Colab Notebooks/DATASETS/GEMINI/INPUT/figurative_errors.csv", index=False)

# Fixed originals

São os de caracteres especiais, entradas numéricas, voids, códigos

## Specials cases that we didn't translate

In [ ]:
path_filtered = "/content/drive/MyDrive/Colab Notebooks/DATASETS/ALPACA_FILTERED/"

df_numbers = pd.read_csv(path_filtered+"df_all_numbers.csv")[['id', 'instance', 'class_type', 'all_number_instance']]
df_numbers['id'] = df_numbers['id'].str.replace(r'(_ins|_inp|_out)$', '', regex=True)
print(df_numbers.shape, df_numbers['all_number_instance'].count(), df_numbers['id'].value_counts().sum())
df_numbers.head()

(2024, 4) 2024 2024


,id,instance,class_type,all_number_instance
0,6,4/16,inp,True
1,37,"[2, 3, 7, 8, 10]",inp,True
2,41,3x+5y=9,inp,True
3,53,"2, 4, 0, 8, 3",inp,True
4,61,101101,inp,True


In [ ]:
df_code = pd.read_csv(path_filtered+"df_code.csv")[['id', 'instance', 'class_type', 'contains_code']]
df_code['id'] = df_code['id'].str.replace(r'(_ins|_inp|_out)$', '', regex=True)
print(f"Code---\n {df_code.shape}, contains_code: {df_code['contains_code'].count()}, ids_uniques: {df_code['id'].nunique()}")
df_code.head()

Code---
 (1009, 4), contains_code: 1009, ids_uniques: 938


,id,instance,class_type,contains_code
0,14905,"Identify any errors in the following Python code.\n\ndef say_hello(name):\n print(""Hello %s!"" % name)\n\nsay_hello(""John"")",ins,True
1,36891,"Given the code snippet, identify and fix any syntax error:\n\ndef findSum(a, b):\n return a+b",ins,True
2,17,"def factorialize(num):\n factorial = 1\n for i in range(1, num):\n factorial *= i\n \n return factorial",inp,True
3,1299,# main.py\n\ndef hello():\n print('Hello World!')\n\nif __name__ == '__main__':\n hello(),inp,True
4,1600,"def add_numbers(x,y):\n return x+y\n\nadd_numbers(2)",inp,True


In [ ]:
df_specials = pd.read_csv(path_filtered+"df_specials.csv")[['id', 'instance', 'class_type', 'only_special_characters']]
df_specials['id'] = df_specials['id'].str.replace(r'(_ins|_inp|_out)$', '', regex=True)
print(f"Specials---\n {df_specials.shape}, only_special_characters: {df_specials['only_special_characters'].count()}, ids_uniques: {df_specials['id'].nunique()}")
df_specials.head()

Specials---
 (11, 4), only_special_characters: 11, ids_uniques: 10


,id,instance,class_type,only_special_characters
0,18545,+,inp,True
1,38726,>,inp,True
2,51186,★★★★★,inp,True
3,51912,🐥,inp,True
4,7165,😆,out,True


In [ ]:
df_voids = pd.read_csv(path_filtered+"df_voids.csv")
df_voids['void'] = True
df_voids['id'] = df_voids['id'].str.replace(r'(_ins|_inp|_out)$', '', regex=True)
df_voids = df_voids[['id', 'instance', 'class_type', 'void']]

print(f"Voids---\n {df_voids.shape}, void: {df_voids['void'].count()}, ids_uniques: {df_voids['id'].nunique()}")
df_voids.head()

Voids---
 (31779, 4), void: 31779, ids_uniques: 31763


,id,instance,class_type,void
0,0,NaN,inp,True
1,1,NaN,inp,True
2,2,NaN,inp,True
3,3,NaN,inp,True
4,4,NaN,inp,True


## Concat the specials case

In [ ]:
not_figurative_concat = pd.concat([df_numbers, df_voids, df_specials, df_code], ignore_index=True)
not_figurative_concat.shape[0] == (df_numbers.shape[0] + df_voids.shape[0] + df_specials.shape[0] + df_code.shape[0])

True

In [ ]:
# Separando cada um de acordo com o seu tipo, para fazer o join de formato original
nf_ins = not_figurative_concat[not_figurative_concat['class_type'] == 'ins'].rename(columns={'instance':'instruction'})[['id', 'instruction']]
nf_ins = nf_ins.astype({'id':int})
nf_inp = not_figurative_concat[not_figurative_concat['class_type'] == 'inp'].rename(columns={'instance':'input'})[['id','input']]
nf_inp = nf_inp.astype({'id':int})
nf_out = not_figurative_concat[not_figurative_concat['class_type'] == 'out'].rename(columns={'instance':'output'})[['id', 'output']]
nf_out = nf_out .astype({'id':int})


df_originals = pd.merge(nf_ins, nf_inp, on='id', how='outer')
df_originals = pd.merge(df_originals, nf_out, on='id', how='outer')
df_originals = df_originals.dropna(subset=['instruction', 'input', 'output'], how='all')
df_originals.columns = df_originals.columns.str.upper()
print(f"Not figurative: {df_originals.shape}\nIds únicos:{df_originals['ID'].nunique()}")
df_originals.head()

Not figurative: (2464, 4)
Ids únicos:2462


,ID,INSTRUCTION,INPUT,OUTPUT
5,6,NaN,4/16,NaN
14,17,NaN,"def factorialize(num):\n factorial = 1\n for i in range(1, num):\n factorial *= i\n \n return factorial","def factorialize(num):\n factorial = 1\n for i in range(num, 0, -1):\n factorial *= i\n \n return factorial"
29,37,NaN,"[2, 3, 7, 8, 10]",NaN
32,41,NaN,3x+5y=9,3x + 5y - 9 = 0
38,53,NaN,"2, 4, 0, 8, 3","0, 2, 3, 4, 8"


In [ ]:
df_originals.shape[0] ==  df_originals.ID.nunique()

False

In [ ]:
df_originals = df_originals.drop_duplicates(subset='ID', keep='first')
df_originals.shape[0] ==  df_originals.ID.nunique()

True

In [ ]:
#Antes de slavar, sempre verificar o shape e os ids únicos
#df_originals.to_csv("/content/drive/MyDrive/Colab Notebooks/DATASETS/ALPACA_FILTERED/result_not_to_figurative_joined.csv", index=False)

#**FAzer depois**

# Join figurative base + originals (not to figurative) - Esse vou juntar só quando terminar a glosa

In [ ]:
print("Input Glosa: "df_to_glosa.head()

In [ ]:
df_to_glosa['TYPE'] = df_to_glosa['ID'].str.extract(r'(ins|inp|out)$')
df_to_glosa['ID'] = df_to_glosa['ID'].str.replace(r'(ins|inp|out)$', '', regex=True)
df_to_glosa.head()

,ID,PT,tokens_est_instance,tokens_est_id,TYPE
0,0,Dê três dicas para se manter saudável.,16,2,ins
1,2,Descreva a estrutura de um átomo.,15,2,ins
2,3,Como podemos reduzir a poluição do ar?,16,2,ins
3,4,Descreva uma ocasião em que você teve que tomar uma decisão difícil.,28,2,ins
4,5,Identifique o estranho.,8,2,ins


In [ ]:
df_ins = df_to_glosa[df_to_glosa['TYPE'] == 'ins'].rename(columns={"PT":"instruction"}).drop(columns='TYPE')[["ID", "instruction"]]
df_inp = df_to_glosa[df_to_glosa['TYPE'] == 'inp'].rename(columns={"PT":'input'}).drop(columns='TYPE')[["ID","input"]]
df_out = df_to_glosa[df_to_glosa['TYPE'] == 'out'].rename(columns={"PT":'output'}).drop(columns='TYPE')[["ID", "output"]]

In [ ]:
df_format = pd.merge(df_ins, df_inp, on='ID', how='outer')
df_format = pd.merge(df_format, df_out, on='ID', how='outer')
df_format.head()

,ID,instruction,input,output
0,0,Dê três dicas para se manter saudável.,NaN,1. Coma uma dieta balanceada e certifique-se de incluir muitas frutas e vegetais. \n2. Exercite-se regularmente para manter seu corpo ativo e forte. \n3. Durma o suficiente e mantenha um horário de sono consistente.
1,10,Como Júlio César morreu?,NaN,"Júlio César foi assassinado por um grupo de até 60 conspiradores, liderados por Caio Cássio Longino e Marco Júnio Bruto, no Senado no dia 15 de março de 44 a.C."
2,100,Crie um banco de dados para registrar os salários dos funcionários.,NaN,"O banco de dados deve conter campos para nome do funcionário, cargo, salário e data. Ele também deve incluir um campo para o gerente do funcionário, para que os salários possam ser alocados adequadamente entre os departamentos. O banco de dados também deve ser capaz de gerar relatórios sobre despesas salariais para departamentos ou indivíduos."
3,1000,Liste 3 eventos históricos relacionados ao seguinte país,NaN,"Os três eventos históricos relacionados ao Canadá são o estabelecimento do Domínio do Canadá em 1867, a adoção da Bandeira Canadense em 1965 e a assinatura do Acordo de Livre Comércio Canadá-EUA em 1988."
4,10000,Cite cinco países do Mediterrâneo.,NaN,"Grécia, Itália, Espanha, Turquia, França"


In [ ]:
df_format.shape, df_format['ID'].nunique()

((52047, 4), 51984)

In [ ]:
# Remove as duplicatas mantendo apenas a primeira ocorrência de cada 'ID'
df_format = df_format.drop_duplicates(subset='ID', keep='first')

# Verificar novamente o formato após a remoção
print("Shape após remoção de duplicatas:", df_format.shape)
print("IDs únicos após remoção:", df_format['ID'].nunique())

Shape após remoção de duplicatas: (51984, 4)
IDs únicos após remoção: 51984


In [ ]:
52002-51984

18

In [ ]:
df_format.to_csv("/content/drive/MyDrive/Colab Notebooks/DATASETS/GEMINI/chat_format_figurative_complete.csv", index=False)